# Projection onto intersection of sets

We try to find the projection of a vector into the intersection of simple sets (whose projection can be computed easily) - See Appendix A of *A Convex Approach to Minimal Partitions Antonin Chambolle, Daniel Cremers, Thomas Pock*


$$
proj_K(x) = \bigcap_{1 \leq i_1 < i_2 \leq k} K_{i_1,i_2} \quad  K_{i_1,i_2}= \{ x: |x_{i_2} - x_{i_1}| \leq \sigma_{i1, i2} \quad \forall i1<i2 \}
$$

In [19]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from pyproximal.projection import *
from pyproximal.proximal import *

np.random.seed(42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Create vector x

In [20]:
k = 3
xtrue = np.random.normal(0, 50, k)
x = xtrue.copy()
x1 = xtrue.copy()
sigma = np.array([[3,2,1], [2,2,1], [3,4,1]])
sigma = sigma.T @ sigma
print(sigma)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.linalg.norm(x[i1] - x[i2]))

[[22 22  8]
 [22 24  8]
 [ 8  8  3]]
0 1 22 31.74892270912087
0 2 8 7.548719254472989
1 2 8 39.297641963593854


Projection of vector x

In [21]:
niter = 40
tol = 1e-20

x12 = np.zeros((k,k))
for iiter in range(niter):
    xold = x.copy()
    for i1 in range(k-1):
        for i2 in range(i1+1, k):
            xtilde = x[i2] - x[i1] + x12[i1, i2]
            xtildeabs = np.abs(xtilde)
            xdtilde = np.maximum(0, xtildeabs - sigma[i1, i2]) * xtilde / xtildeabs
            x[i1] = x[i1] + 0.5 * (xdtilde - x12[i1, i2])
            x[i2] = x[i2] - 0.5 * (xdtilde - x12[i1, i2])
            x12[i1, i2] = xdtilde
    if max(np.abs(x - xold)) < tol:
        break
print(iiter)

25


Check projected vector satisfy the condition

In [22]:
for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(xtrue[i1] - xtrue[i2]))
        
for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

0 1 22 31.74892270912087
0 2 8 7.548719254472989
1 2 8 39.297641963593854
0 1 22 16.000000000000004
0 2 8 8.0
1 2 8 8.000000000000004


In [23]:
ic = IntersectionProj(k, 1, sigma, niter, tol)
x1 = ic(x1)
x1 - xtrue

array([ -0.06673448,  15.68218822, -15.61545374])

In [24]:
ic = Intersection(k, 1, sigma, niter, tol)
print(ic(xtrue))
x = ic.prox(xtrue, 1)
print(ic(x))

False
True


Repeat the same, now with a matrix with n columns (algorithm works on each column indipendently)

In [25]:
k = 3
n = 5
xtrue = np.random.normal(0, 50, (k, n))
x = xtrue.copy()
x1 = xtrue.copy()
sigma = np.array([[3,2,1], [2,2,1], [3,4,1]])
sigma = sigma.T @ sigma
print(sigma)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

[[22 22  8]
 [22 24  8]
 [ 8  8  3]]
0 1 22 [ 99.62521212  38.83567092  11.46403679 102.24712845  26.27362288]
0 2 8 [171.81550505  74.53822289  16.40752861 129.60219679  22.65936983]
1 2 8 [ 72.19029294 113.3738938    4.94349182  27.35506834   3.61425305]


In [26]:
niter = 50
tol = 1e-20

x12 = np.zeros((k,k,n))
for iiter in range(niter):
    xold = x.copy()
    for i1 in range(k-1):
        for i2 in range(i1+1, k):
            xtilde = x[i2] - x[i1] + x12[i1, i2]
            xtildeabs = np.abs(xtilde)
            xdtilde = np.maximum(0, xtildeabs - sigma[i1, i2]) * xtilde / xtildeabs
            x[i1] = x[i1] + 0.5 * (xdtilde - x12[i1, i2])
            x[i2] = x[i2] - 0.5 * (xdtilde - x12[i1, i2])
            x12[i1, i2] = xdtilde
    if max(np.sum(np.abs(x-xold), axis=0)) < tol:
        break
print(iiter)

36


In [27]:
for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

0 1 22 [1.60000000e+01 3.55271368e-15 7.26027249e+00 1.60000000e+01
 1.60000000e+01]
0 2 8 [8. 8. 8. 8. 8.]
1 2 8 [8.         8.         0.73972751 8.         8.        ]


Same using the projection operator in PyProximal

In [28]:
ic = IntersectionProj(k, n, sigma, niter, tol)
x1 = ic(x1)
x1 = x1.reshape(k,n)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x1[i1] - x1[i2]))

0 1 22 [1.60000000e+01 2.25490737e-11 7.26027249e+00 1.60000000e+01
 1.60000000e+01]
0 2 8 [8. 8. 8. 8. 8.]
1 2 8 [8.         8.         0.73972751 8.         8.        ]


and the proximal operator in PyProximal

In [29]:
ic = Intersection(k, n, sigma, niter, tol)
print(ic(xtrue, 1e-3))
x = ic.prox(xtrue, 1)
print(ic(x, 1e-3))
x = x.reshape(k,n)

for i1 in range(k-1):
    for i2 in range(i1+1, k):
        print(i1, i2, sigma[i1, i2], np.abs(x[i1] - x[i2]))

False
True
0 1 22 [1.60000000e+01 2.25490737e-11 7.26027249e+00 1.60000000e+01
 1.60000000e+01]
0 2 8 [8. 8. 8. 8. 8.]
1 2 8 [8.         8.         0.73972751 8.         8.        ]
